In [1]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
from PIL import ImageGrab
import os
import time
import numpy as np
import pandas as pd
import psycopg2
from pgvector.psycopg2 import register_vector
from psycopg2.extras import execute_values
from dotenv import load_dotenv
load_dotenv()

d:\LocalRecall\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
## I know its local, so we dont need to worry about the security of the key, but i will use the .env file to store the key anyway
def getConections():
    conn = psycopg2.connect(
        host="localhost",
        port="5432",
        database="postgres",
        user= os.getenv('user'),
        password= os.getenv('password')
    )

    cur = conn.cursor()
    cur.execute('CREATE EXTENSION IF NOT EXISTS vector')
    register_vector(conn)
    return conn, cur
conn, cur = getConections()

In [20]:
#FOR COMPARISION
def getIndex():
    from pinecone import Pinecone
    pc = Pinecone(api_key=os.getenv('pineKey'))
    index = pc.Index("vectors")
    return index
index = getIndex()

In [5]:
def getProcessor():
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
    return processor

def getProcImage(imgs: np.ndarray):
    processor = getProcessor()
    size = {'height': 224, 'width': 224}
    proc_img = processor(images=imgs, size = size , return_tensors="pt")
    return proc_img


# while True:
#     s1 = ImageGrab.grab(); s1 = np.asarray(s1)
#     s2 = ImageGrab.grab(); s2 = np.asarray(s2)

#     imgs = np.array([s1, s2])
#     procImages = getProcImage(imgs)
#     s1p = procImages.get('pixel_values')[0]
#     s2p = procImages.get('pixel_values')[1]

#     #Rudamentary difference check
#     # May need to implement a more robust method like PySceneDetect
#     diff = torch.abs(s1p - s2p).sum()
    
#     if int(diff) > 2000:
#         s2 = Image.fromarray(s2)
#         s2.save(f'snaps/{time.time()}.png')


In [6]:
def prepMeta(dir : str = 'snaps'):
    files = os.listdir(dir)
    meta = []
    for file in files:
        meta.append(file.split('.png')[0].strip())
    return meta

def getClipModel():
    return CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
s = time.time() 
model = getClipModel()
print(time.time() - s)


def getEmbeddings(dirMode: bool = False, imageTensors: np.ndarray = None, model = None):
    model = getClipModel() if model is None else model
    if dirMode:
        meta = prepMeta(dir = 'snaps')
        imageNdarray = []
        imageNdarray.extend([Image.open(f'snaps/{m}.png') for m in meta])
        batch_tensor = getProcImage(imageNdarray)
        batch_tensor = batch_tensor.get('pixel_values')
          
    else:
        if imageTensors is None:
            raise Exception('No images found in array')
        batch_tensor = torch.stack(imageTensors)

    inputs = {'pixel_values': batch_tensor}
        
    with torch.no_grad():
        output = model.get_image_features(**inputs)

    documents = []
    for i, m in enumerate(meta):
        documents.append({'id': m, 'vector': output[i].tolist()})
    return documents

documents = getEmbeddings(dirMode = True, model = model)
df = pd.DataFrame(documents)
df.to_csv('embeddings.csv', index = False)

16.53878688812256


In [8]:
def createTable(name = 'vectors'):
    cur.execute(f'''CREATE TABLE IF NOT EXISTS {name}(
        id VARCHAR PRIMARY KEY,
        ts float8,
        vector vector(512)
    )''')
    conn.commit()
createTable("vectors")

In [9]:
def insertVectors(documents, insertOne = False):
    if insertOne:
        cur.execute("INSERT INTO vectors (id, vector, ts) VALUES (%s, %s, %s)", (documents.get('id'), documents.get('vector'), documents.get('id')))
        conn.commit()

    data_list = [(doc.get('id'), doc.get('vector'), doc.get('id')) for doc in documents]
    execute_values(cur, "INSERT INTO vectors (id, vector, ts) VALUES %s", data_list)
    conn.commit()

# conn, cur = getConections()
insertVectors(documents)

In [11]:
pineConeDocs = [{'id': doc.get('id'), 'values': doc.get('vector')} for doc in documents]
index.upsert(pineConeDocs)
index.describe_index_stats()

{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10}},
 'total_vector_count': 10}

In [3]:
def getAllVectors(ids = None):
    if ids is None:
        cur.execute("SELECT * FROM vectors")
        
    elif isinstance(id, list):
        cur.execute(f"SELECT * FROM vectors WHERE id IN {tuple(id)}")
    rows = cur.fetchall()
    
    return rows
conn, cur = getConections()
res = getAllVectors()
res

[('1231232321.3231213',
  1231232321.3231213,
  array([ 7.24290252e-01, -1.09977853e+00,  3.32713932e-01,  2.97519028e-01,
         -7.42908865e-02, -4.70190942e-02,  3.23487848e-01,  3.60316694e-01,
         -6.56362772e-02, -1.92416817e-01,  1.98476881e-01, -8.64450634e-02,
          1.13011271e-01, -2.46686712e-01, -1.55880712e-02,  2.30846509e-01,
          6.63077831e-02,  2.63048053e-01, -1.04203641e-01,  6.75864592e-02,
         -2.18285412e-01, -1.71318352e-02,  3.27199191e-01,  8.82261992e-02,
         -1.07569292e-01, -3.73483926e-01, -7.22235978e-01,  3.35365117e-01,
         -1.82774797e-01,  2.40101576e-01, -4.24170494e-03, -1.96110591e-01,
         -1.83883548e-01, -9.79574025e-02, -1.18310966e-01, -1.72706693e-01,
          1.55748904e-01,  4.49823290e-02,  3.18331003e-01,  3.31071973e-01,
          1.15846835e-01, -1.27576515e-01, -8.98856968e-02, -1.85617954e-01,
         -3.38840038e-02, -5.31819582e-01,  5.23297489e-01,  2.28222102e-01,
          4.24122624e-02,  3.8

In [17]:
def dotQuery(inputText:str = None, ids:list = None, topk = 1, includeDistance = False, 
             includeVector = False, metadataFilter:list = None, inId:list = None, pineCone = False):
    
    if inputText is None:
        raise Exception('No input text found')
    
    processor = getProcessor()
    inputs = processor(text = inputText, return_tensors="pt", padding=True)
    text_features = model.get_text_features(**inputs)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    text_features = text_features.tolist()[0]
    
    if pineCone:

        pineres = index.query(vector = text_features, top_k = topk, 
                          include_values = includeVector, 
                          include_metadata= True if metadataFilter else False)

    base_query = "SELECT "
    
    # Select fields based on includeDistance and includeVector
    fields = []
    if includeVector:
        fields.append("id, vector")
    else:
        fields.append("id")
    
    if includeDistance:
        # Cosine similarity, higher is closer
        fields.append(f" 1 - (vector <=> '{text_features}'::vector) AS cosine_similarity")
    
    if metadataFilter:
        fields.extend(metadataFilter)
    
    base_query += ", ".join(fields) + " FROM vectors"

    if inId is not None:
        if len(inId) == 1:
            in_clause = f"WHERE id = '{inId[0]}'"
        else:
            in_clause = f"WHERE id IN {tuple(inId)}"
        base_query += " " + in_clause
    
    # Order by distance
    base_query += f" ORDER BY vector <=> '{text_features}'::vector LIMIT {topk}"

    query = base_query
    
    cur.execute(query)
    x = cur.fetchall()

    if includeDistance:
        x = sorted(x, key=lambda x: x[1], reverse=True)

    if pineCone:
        res = {
            'pgVector': x,
            'pineCone': pineres
        }
        return res
    
    return {
        'pgVector': x
    }

conn, cur = getConections()
dotQuery(inputText = "snapshot of a white screen with blue and black text ", topk = 2, includeDistance = True, includeVector = False, pineCone = False)


{'pgVector': [('1231232321.3231213', 0.24724543627049256),
  ('1716655832.8881292', 0.24413781816879054)]}

In [ ]:
# cur.execute("DELETE FROM vectors")
# conn.commit()